# Linux_elasticsearch

## 간단한 예제
- 생성 <br>
PUT my_index/_doc/1
{
    "name": "chang ho Kim",
    "message": "Hi elasticsearch" 
}

- 검색 <br>
GET my_index/_doc/1

- 삭제 <br>
DELETE my_index/_doc/1
<br>
or <br>
DELETE my_index

- 추가 덮어쓰기 <br>
PUT my_index/_doc/1
{
    "age": 40
}
<br>
- 추가 해당 내용만 수정 또는 추가 <br>
PUT my_index/_update/1
{
    "doc":{
        "age": 40
    }
}

- bulk test <br>
POST _bulk
{"index": {"_index": "test", "_id": "1"}}
{"field": "value one"}
{"index": {"_index": "test", "_id": "2"}}
{"field": "value two"}
{"delete": {"_index": "test", "_id": "2"}}
{"create": {"_index": "test", "_id": "3"}}
{"field": "value three"}
{"update": {"_index": "test", "_id": "1"}}
{"doc": {"field": "value two"}}
<br>
GET test/_doc/1
<br>
GET test/_doc/2
<br>
GET test/_doc/3
<br>

- 데이터 검색 <br>
GET test/_search?q=two
<br>
GET test/_search?q=three
<br>
GET test/_search?q=field:three AND field: value
<br>
GET test/_search
{
    "query": {
        "match": {
            "message": "Hi!!!"
        }
    }
}

## logstash : 실시간 파이프라인 기능을 가진 오픈소스 데이터 수집 엔진
- download (1번 노드에서 진행) <br>
wget https://artifacts.elastic.co/downloads/logstash/logstash-7.14.1-linux-x86_64.tar.gz
<br>
tar xfz logstash-7.14.1-linux-x86_64.tar.gz
<br>
- 실행 시, 키보드 입력값 반환
~/logstash-7.14.1/bin/logstash -e 'input { stdin { } } output { stdout { } }'
<br>
- 파일 생성 <br>
cd ~/logstash-7.14.1/config
<br>
gedit test.conf
<br>
- 파일 내용 입력 <br>
input { stdin{}}
output { stdout{}}
- 파일 저장 후 아래 명령어 실행 <br>
~/logstash-7.14.1/bin/logstash -f test.conf


- 파일 수정 (9900 포트에서 들어오는 입력값 받기) <br>
input { 
	tcp {
		port => 9900
	}
}
output { 
	stdout {}
}

- 재실행 <br>
~/logstash-7.14.1/bin/logstash -f test.conf

- 새로운 터미널에서 문자열 입력 <br>
echo 'hello logstash' | nc localhost 9900

- 파일 수정 (9900 포트에서 들어오는 입력값 elasticsearch로 보내기) <br>
input { 
	tcp {
		port => 9900
	}
}
output { 
	elasticsearch {
		hosts => ["localhost:9200"]
		user => "elastic"
		password => "bigdata"		
	}
}

- 재실행 <br>
~/logstash-7.14.1/bin/logstash -f test.conf

- 새로운 터미널에서 문자열 입력 <br>
echo 'hello logstash' | nc localhost 9900

- 3번 노드 키바나 실행 후, dev tools <br>
GET _cat/indices
GET logstash-*/_search

- weblog 파일 이용하기 1 (파일 업로드 & 확인) <br>
head ~/weblog-sample.log

- 파일 수정 <br>
input { 
	tcp {
		port => 9900
	}
}
filter {
    grok {
        match => {"message" => "%{COMBINEDAPACHELOG}"}
    }
}
output { 
    stdout {}
}


- 재실행 <br>
~/logstash-7.14.1/bin/logstash -f test.conf

- 새로운 터미널에서 문자열 입력 <br>
head -n 1 ~/weblog-sample.log | nc localhost 9900

- 노드3에서 기존 생성 인덱스 삭제 <br>
DELETE logstash*

- weblog 파일 이용하기 2 <br>
input { 
	tcp {
		port => 9900
	}
}
filter {
	grok {
	        match => {"message" => "%{COMBINEDAPACHELOG}"}
	}
	geoip {
		source => "clientip"
	}
	useragent {
		source => "agent"
		target => "useragent"
	}
	mutate {
		convert => {
			"bytes" => "integer"
		}
	}
	date {
		match => ["timestamp", "dd/MMM/yyyy:HH:mm:ss Z"]
		target => "logdate"
	}
}
output { 
	elasticsearch {
		hosts => ["localhost:9200"]
		user => "elastic"
		password => "bigdata"		
	}
}
<br>

- 재실행 <br>
~/logstash-7.14.1/bin/logstash -f test.conf

- 새로운 터미널에서 문자열 입력 <br>
head -n 1 ~/weblog-sample.log | nc localhost 9900

- 3번 노드 키바나 실행 후, dev tools <br>
GET _cat/indices
GET logstash-*/_search

## 한글 형태소 분석기
- 형태소 분석기 다운로드 & 재부팅 & elasticsearch 실행(모든 노드) <br>
~/es-714/bin/elasticsearch-plugin install analysis-nori

- data load 확인 <br>
curl "elastic-1:9200/_cat/nodes?v" -u elastic:bigdata

- 한글 형태소 분석
GET _analyze
{
    "tokenizer": "standard",
    "text": ["동해물과 백두산이"]
}

- my_nori 인덱스에 "해물" 사전을 추가한 my_nori_tokenizer 생성<br>
PUT my_nori
{
    "settings": {
        "analysis": {
            "tokenizer": {
                "my_nori_tokenizer": {
                    "type": "nori_tokenizer",
                    "user_dictionary_rules":[
                        "해물"
                    ]
                }
            }
        }
    }
}
<br>
- my_nori_tokenizer 토크나이저로 "동해물과" 분석 <br>
GET my_nori/_analyze
{
    "tokenizer": "my_nori_tokenizer",
    "text": [
        "동해물과"
    ]
}
<br>

DELETE my_nori <br>
PUT my_nori
{
    "settings": {
        "analysis": {
            "tokenizer": {
                "nori_none": {
                    "type": "nori_tokenizer",
                    "decompound_mode": "none"
                },
                "nori_discard": {
                    "type": "nori_tokenizer",
                    "decompound_mode": "discard"
                },
                "nori_mixed": {
                    "type": "nori_tokenizer",
                    "decompound_mode": "mixed"
                }
            }
        }
    }
}
<br>
GET my_nori/_analyze
{
    "tokenizer": "nori_none",
    "text": ["백두산이"]
}
<br>
GET my_nori/_analyze
{
    "tokenizer": "nori_discard",
    "text": ["백두산이"]
}
<br>
GET my_nori/_analyze
{
    "tokenizer": "nori_mixed",
    "text": ["백두산이"]
}

- 수사(숫자) 인식 후 필터링 <br>
PUT my_pos
{
    "settings": {
        "index": {
            "analysis": {
                "filter": {
                    "my_pos_f": {
                        "type": "nori_part_of_speech",
                        "stoptags": [
                            "NR"
                        ]
                    }
                }
            }
        }
    }
}
<br>
- my_pos_f 토큰필터로 "다섯아이가" 분석 <br>
GET my_pos/_analyze
{
    "tokenizer": "nori_tokenizer",
    "filter": [
        "my_pos_f"
    ],
    "text": ["다섯아이가"]
}

## filebeat 설치 (노드1에서) : 대용량 파일 업로드시 사용
- file 다운로드 <br>
wget https://artifacts.elastic.co/downloads/beats/filebeat/filebeat-7.14.1-linux-x86_64.tar.gz
<br>

tar xfz filebeat-7.14.1-linux-x86_64.tar.gz
<br>
mv filebeat-7.14.1-linux-x86_64.tar.gz files
<br>
mv filebeat-7.14.1-linux-x86_64 fb-714
<br>

- file 수정 : 문서의 각 부분 수정<br>
gedit ~/fb-714/filebeat.yml <br>

<!-- # ============================== Filebeat inputs =============================== -->

<!-- filebeat.inputs:

- type: log -->

  <!-- # Change to true to enable this input configuration. -->
  enabled: true

  <!-- # Paths that should be crawled and fetched. Glob based paths. -->
  paths:
    - /home/elastic-1/weblog-sample.log
    <!-- #- c:\programdata\elasticsearch\logs\* -->
...
<!-- # ================================== Outputs =================================== -->

<!-- # Configure what output to use when sending the data collected by the beat.

# ---------------------------- Elasticsearch Output ----------------------------
output.elasticsearch:
  # Array of hosts to connect to.
  hosts: ["localhost:9200"]

  # Protocol - either `http` (default) or `https`.
  #protocol: "https"

  # Authentication credentials - either API key or username/password.
  #api_key: "id:api_key" -->
  username: "elastic"
  password: "bigdata"
<br>
- filebeat 실행 <br>
cd fb-714 <br>
./filebeat

- message 필드 확인 <br>
GET _cat/indices 
<br>
GET filebeat-7.14.1*/_search
<br>

- filebeat 중지 후 확인 <br>
GET filebeat-7.14.1*/_count
<br>
DELETE filebeat*


## Kibana로 데이터 직접 import
- upload
- name 지정 (pattern no check)
- data type 수정 (확진자: long -> integer, time: byte)
- import 
- data 확인 <br>
GET /covid-kr-time
<br>
GET /covid-kr-time/_search
<br>
- Stack management -> Index pattern -> create index pattern -> "covid-kr-time" -> @timestamp
- Discover -> 날짜 (data file: 2020년도) -> 데이터 확인
- Visualize Data -> Create Visualize Data -> Aggregation based -> Line (선택) -> Data 선택
- Dashboard -> Create Dashboard -> Add from library (기존에 만든 시각화 자료 활용)

# 결론적으로 정적이지 않고 (python), 동적인 프로그램이기에 효용성이 더 높음